# Esercizio 5

Prendere in input un file in formato `GTF` (Gene Transfer Format) e il file `FASTA` della genomica di riferimento.

In dipendenza di una determinata scelta, ricostruire tutte le sequenze di un certo tipo (trascritto o coding sequence o 5'UTR o 3'UTR) che sono annotate nel file GTF e salvarle, in formato FASTA, in un file.

L'*header* `FASTA` di ogni sequenza prodotta deve contenere:

- il nome del gene di riferimento
- l’identificatore del trascritto di riferimento
- la lunghezza della sequenza
- il tipo di sequenza (trascritto oppure CDS oppure 5'UTR oppure 3'UTR)
- lo *strand*
    
Esempio di *header* per un trascritto:
         
    >ARHGAP4; U52112.4-003; len=3235 type=exon; strand=-

Esempio di *header* per una CDS:

    >AVPR2; U52112.2-003; len=642; type=CDS; strand=+
    
Se si sceglie di ricostruire le *coding sequences*, produrre in standard output la loro separazione in codoni.
   
***


## Dataset in input

In [4]:
gtf_file_name = './input.gtf'
reference_file_name = './ENm006.fa'

## Tipo di sequenza da ricostruire

Valori possibili:

- `exon`, se si vogliono ricostruire i trascritti
- `CDS`, se si vogliono ricostruire le coding sequences
- `5UTR`, se si vogliono ricostruire le 5'UTR
- `3UTR`, se si vogliono ricostruire le 3'UTR

In [5]:
selected_feature = 'CDS'

## Importare il modulo `re`

In [6]:
import re

## Definire la funzione per estrarre il gene e il trascritto associati a una certa *feature*

La funzione deve prendere come argomento un *record* GTF e restituire la tupla `(gene_name, transcript_id)` contenente il nome del gene e l'identificatore del trascritto associati al *record*.

**NOTA BENE**: gli attributi all'interno del nono campo di un file `GTF` non hanno ordine fisso all'interno del campo.

In [22]:
def get_gene_and_transcript(gtf_record):
    gene_name = re.search(r'gene_id\s"(.+?)";', gtf_record).group(1)
    transcript_id = re.search(r'transcript_id\s"(.+?)";', gtf_record).group(1)
    
    return (gene_name, transcript_id)

## Definire la funzione che formatta una sequenza in `FASTA` 

Fare riferimento all'esercizio 3.

In [8]:
def format_fasta(header, sequence, record = 80):    
    p = re.compile('\w{,' + str(record) + '}')
    return header + '\n' + '\n'.join(re.findall(p, sequence))

## Definire la funzione per restituire il *reverse and complement* di una sequenza nucleotidica

La funzione deve prendere come argomento una sequenza nucleotidica e restituire il *reverse and complement* della sequenza in lettere maiuscole.

In [9]:
def reverse_complement(nucleotide_sequence):
    complement_dict = {'A' : 'T', 'T' : 'A', 'C' : 'G', 'G' : 'C'}
    
    return ''.join(complement_dict[b] for b in nucleotide_sequence[::-1].upper())

In [10]:
reverse_complement('AAACCC')

'GGGTTT'

## Definire la funzione che ricostruisce la sequenza di un certo tipo

La funzione deve prendere i seguenti argomenti:
- la lista delle *features* che compongono la sequenza da ricostruire
- la sequenza della genomica di riferimento
- lo *strand*

Prevedere che ogni *feature* nella lista sia una tupla `(start, end, frame)` di dimensione tre, in cui i primi due valori sono le posizioni 1-based di inizio e fine della *feature* sulla genomica di riferimento. Il terzo elemento `frame` è diverso da `.` (dot) solo se si sta ricostruendo una coding sequence.

In [11]:
def reconstruct_sequence(feature_list, reference_sequence, strand):
    sequence = ''
    feature_list.sort()
    for x in feature_list:
        sequence = sequence + reference_sequence[x[0]-1: x[1]]
        
    if strand == '-':
        return reverse_complement(sequence)
    else:
        return sequence

## Recuperare la genomica di riferimento

In [12]:
with open(reference_file_name, 'r') as input_file:
    reference_sequence = ''.join(input_file.readlines()[1:]).replace('\n', '')

In [13]:
reference_sequence

'...'

## Ottenere dal file GTF i *record* che servono per ricostruire le sequenze del tipo scelto

In [16]:
with open(gtf_file_name, 'r') as input_file:
    selected_gtf_records = [record for record in input_file.readlines() if record.split('\t')[2] == selected_feature]

In [17]:
selected_gtf_records

['ENm006\tVEGA_Known\tCDS\t71783\t71788\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t70312\t70440\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t69989\t70210\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64935\t65036\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64566\t64673\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64385\t64459\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72761\t72963\t.\t-\t0\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72521\t72683\t.\t-\t1\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72253\t72315\t.\t-\t0\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t71872\t71965\t.\t-\t0\ttranscript_id "U52112.4

## Costruire il dizionario degli *strand*

Solo per i geni per cui è stata annotata una sequenza del tipo scelto, creare il dizionario:

- **chiave**: nome del gene
- **valore**: *strand* del gene (rispetto al *reference*)

In [25]:
strand_dict = {}

for record in selected_gtf_records:
    strand = record.split('\t')[6]
    
    (gene_name, transcript_id) = get_gene_and_transcript(record)
    
    strand_dict[gene_name] = strand

In [26]:
strand_dict

{'ARHGAP4': '-', 'AVPR2': '+'}

## Ricostruire le sequenze del tipo scelto

a) Costruire il dizionario degli identificatori dei trascritti:

   - **chiave**: nome del gene
   - **valore**: insieme degli identificatori dei trascritti per cui è annotata la sequenza del tipo scelto

In [27]:
gene_dict = {}

for record in selected_gtf_records:
    (gene_name, transcript_id) = get_gene_and_transcript(record)
    
    value = gene_dict.get(gene_name, set())
    value.add(transcript_id)
    gene_dict[gene_name] = value

In [28]:
gene_dict

{'ARHGAP4': {'U52112.4-001',
  'U52112.4-003',
  'U52112.4-005',
  'U52112.4-010',
  'U52112.4-011',
  'U52112.4-017',
  'U52112.4-019',
  'U52112.4-020',
  'U52112.4-024'},
 'AVPR2': {'U52112.2-001', 'U52112.2-003'}}

b) Costruire il dizionario delle *features* per trascritto:

   - **chiave**: identificatore del trascritto
   - **valore**: lista delle *features*, associate al trascritto, che compongono la sequenza del tipo scelto
   
Ogni *feature* nella lista deve essere rappresentata dalla tupla `(start, end, frame)` contenente le posizioni 1-based di inizio e fine della *feature* sul *reference* e il valore di *frame* della *feature* (che sarà un *dot* - assenza di informazione - nel caso si scelga di ricostruire sequenze diverse dalle *coding sequences*.

In [35]:
composition_dict = {}

for record in selected_gtf_records:
    (gene_name, transcript_id) = get_gene_and_transcript(record)
    
    value = composition_dict.get(transcript_id, [])
    
    fields = record.split('\t')
    
    tupla = (fields[3], fields[4], fields[7])
    value.append(tupla)
    
    composition_dict[transcript_id] = value

In [36]:
composition_dict

{'U52112.4-005': [('71783', '71788', '0'),
  ('70312', '70440', '0'),
  ('69989', '70210', '0'),
  ('64935', '65036', '0'),
  ('64566', '64673', '0'),
  ('64385', '64459', '0')],
 'U52112.4-019': [('72761', '72963', '0'),
  ('72521', '72683', '1'),
  ('72253', '72315', '0'),
  ('71872', '71965', '0')],
 'U52112.4-017': [('72761', '72963', '0'),
  ('72521', '72683', '1'),
  ('72253', '72315', '0'),
  ('71865', '71965', '0')],
 'U52112.4-010': [('63857', '63942', '2'),
  ('62286', '62346', '0'),
  ('61857', '61991', '2'),
  ('61663', '61768', '2'),
  ('61328', '61561', '1'),
  ('61169', '61242', '1'),
  ('60898', '61081', '2')],
 'U52112.4-020': [('72521', '72560', '1'),
  ('71783', '71965', '0'),
  ('70724', '70843', '0'),
  ('70312', '70440', '0'),
  ('70097', '70210', '0')],
 'U52112.4-003': [('77293', '77359', '0'),
  ('72761', '72965', '2'),
  ('72521', '72683', '1'),
  ('72253', '72315', '0'),
  ('71783', '71965', '0'),
  ('70312', '70440', '0'),
  ('69989', '70210', '0'),
  ('6493

c) Se si stanno ricostruendo coding sequences, ottenere (dal precedente) il dizionario dei valori di *frame* della prima *feature* CDS:

   - **chiave**: identificatore di trascritto
   - **valore**: *frame* della prima *feature* CDS

In [39]:
frame_dict = {}

if selected_feature == 'CDS':
    
    for gene_name in gene_dict:
        for transcript_id in gene_dict[gene_name]:
            feature_list = composition_dict[transcript_id]
            sorted_list = sorted(feature_list)
                
            if strand_dict[gene_name] == '+':
                frame = sorted_list[0][2]
            else:
                frame = sorted_list[-1][2]
                    
            frame_dict[transcript_id] = frame

In [40]:
frame_dict

{'U52112.4-010': '2',
 'U52112.4-019': '0',
 'U52112.4-001': '0',
 'U52112.4-017': '0',
 'U52112.4-011': '0',
 'U52112.4-020': '1',
 'U52112.4-003': '0',
 'U52112.4-024': '0',
 'U52112.4-005': '0',
 'U52112.2-001': '0',
 'U52112.2-003': '0'}

d) Ricostruire le sequenze e salvarle in un file `FASTA`

Esempio di *header*:

    >ARHGAP4; U52112.4-003; len=3235; type=[exon|CDS]; strand=-
    
**NB**: se si stanno ricostruendo le *coding sequences*, stampare anche in standard output la separazione in codoni tenendo conto del *frame* della prima *feature*.